### Описание задания:

- Использовать датасет ml-latest.
- Выбрать подходящий подход к построению гибридным системам для предоставления рекомендаций.
- Написать свою гибридную систему

In [1]:
import wget
import zipfile

import pandas as pd

In [4]:
dataset = 'ml-latest'
url = f'https://files.grouplens.org/datasets/movielens/{dataset}.zip'

In [5]:
wget.download(url, f'{dataset}.zip')

100% [......................................................................] 277113433 / 277113433

'ml-latest.zip'

In [7]:
with zipfile.ZipFile(f'{dataset}.zip', 'r') as zip_data:
    zip_data.extractall()

In [8]:
movies = pd.read_csv(f'./{dataset}/movies.csv')
movies.name = 'movies'
ratings = pd.read_csv(f'./{dataset}/ratings.csv')
ratings.name = 'ratings'
tags = pd.read_csv(f'./{dataset}/tags.csv')
tags.name = 'tags'

In [9]:
def get_analises(dataset) -> None:
    print(dataset.name)
    dataset.info()
    print(f'Дублирующих записей: {dataset.duplicated().sum()}')
    print('------------')

### Первичный анализ данных

In [10]:
get_analises(movies)
get_analises(ratings)
get_analises(tags)

movies
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB
Дублирующих записей: 0
------------
ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB
Дублирующих записей: 0
------------
tags
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108997 non-null  int64 
 1   movieId    1108